# Machine Learning Engineer Nanodegree
## Capstone Project - Street View House Numbers
Thomas Wieczorek
January 1st, 2017

## I. Definition
_(approx. 1-2 pages)_

### Project Overview
In this section, look to provide a high-level overview of the project in layman’s terms. Questions to ask yourself when writing this section:
- _Has an overview of the project been provided, such as the problem domain, project origin, and related datasets or input data?_
- _Has enough background information been given so that an uninformed reader would understand the problem domain and following problem statement?_

This capstone-project aims to solve the problem to correctly classify housenumber from Street View images. A rule-based approach ("Look for a certain color or pattern...) is not very promising, because the house numbers are very heterogeneously. They come in all colors, different fonts, sometimes they are even one underneath the other.  

![image alt >](res/Street_View.png)
<center>_1. Example of Google Street View in Berlin_</center>
![image alt >](res/Street_View_close.png)
<center>_2. Close up of House Number_</center>

The dataset was taken from Stanford University, which they describe as follows: _SVHN is a real-world image dataset for developing machine learning and object recognition algorithms with minimal requirement on data preprocessing and formatting. It can be seen as similar in flavor to MNIST (e.g., the images are of small cropped digits), but incorporates an order of magnitude more labeled data (over 600,000 digit images) and comes from a significantly harder, unsolved, real world problem (recognizing digits and numbers in natural scene images). SVHN is obtained from house numbers in Google Street View images._ (from http://ufldl.stanford.edu/housenumbers/)


To solve this Computer Vision problem, the domain of Machine Learning is advisable. Especially Deep Learning has been very successful for solving Computer Vision problems like these in the latest years. 


### Problem Statement
In this section, you will want to clearly define the problem that you are trying to solve, including the strategy (outline of tasks) you will use to achieve the desired solution. You should also thoroughly discuss what the intended solution will be for this problem. Questions to ask yourself when writing this section:
- _Is the problem statement clearly defined? Will the reader understand what you are expecting to solve?_
- _Have you thoroughly discussed how you will attempt to solve the problem?_
- _Is an anticipated solution clearly defined? Will the reader understand what results you are looking for?_

The problem statement is to identify which house number is shown in the picture. The images from the SVHN have been resized to a fixed resolution of 32-by-32 pixels. The file type is .mat. In addition to the images, we have the correct labels of every image.

However, if there are more than one number, the images are cropped, so that every number is in the middle of the picture. So the classifier only has to classify one digit.

![image alt >](res/Classification_Examples.png)
<center>_3. Example of correct predictions._</center>

![image alt >](res/Example_Cropping.png)
<center>_4. Example of Cropping a 4-digit house number._</center>

### Metrics
In this section, you will need to clearly define the metrics or calculations you will use to measure performance of a model or result in your project. These calculations and metrics should be justified based on the characteristics of the problem and problem domain. Questions to ask yourself when writing this section:
- _Are the metrics you’ve chosen to measure the performance of your models clearly discussed and defined?_
- _Have you provided reasonable justification for the metrics chosen based on the problem and solution?_
<br><br>

The metrics used for this project were **Accuracy** and the **Confusion Matrix**. The Formula for the **Accuracy** is:

\begin{align}
{Accuracy} & = \frac{tp+tn}{tp+tn+fp+fn} \\\\
{tp} & : {True Positive} \\
{tn} & : {True Negative} \\
{fp} & : {False Positive} \\
{fn} & : {False Negative} \\ 
\end{align}
<br><br>
For my research, I plotted the Accuracy of the **Training Data**, as well as of the **Validation Data** to prevent [Overfitting](https://www.wikiwand.com/en/Overfitting). 

![image alt >](res/Accuracy.png)
<center>_4. Accuracy of Train-Data and Validation-Data._</center>
<br>
If the validation line would be declining while the training line still increasing, it is a very good sign to detect Overfitting. See this figure:

![image alt >](res/Overfitting.png)
<center>_5. Overfitting example_</center>
<br><br>

In addition to this, I always used a Confusion Matrix. This helps to identify problems with certain classes (for example the number 3 always get mixed up with 8, because of the similarity). 

![image alt >](res/ConfusionMatrix.png)
<center>_6. Confusion Matrix_</center>
<br><br>

## II. Analysis
_(approx. 2-4 pages)_



### Data Exploration
In this section, you will be expected to analyze the data you are using for the problem. This data can either be in the form of a dataset (or datasets), input data (or input files), or even an environment. The type of data should be thoroughly described and, if possible, have basic statistics and information presented (such as discussion of input features or defining characteristics about the input or environment). Any abnormalities or interesting qualities about the data that may need to be addressed have been identified (such as features that need to be transformed or the possibility of outliers). Questions to ask yourself when writing this section:
- _If a dataset is present for this problem, have you thoroughly discussed certain features about the dataset? Has a data sample been provided to the reader?_
- _If a dataset is present for this problem, are statistics about the dataset calculated and reported? Have any relevant results from this calculation been discussed?_
- _If a dataset is **not** present for this problem, has discussion been made about the input space or input data for your problem?_
- _Are there any abnormalities or characteristics about the input space or dataset that need to be addressed? (categorical variables, missing values, outliers, etc.)_

![image alt >](res/Examples_Housenumbers/0.png)
![image alt >](res/Examples_Housenumbers/1.png)
![image alt >](res/Examples_Housenumbers/2.png)
![image alt >](res/Examples_Housenumbers/3.png)
![image alt >](res/Examples_Housenumbers/4.png)
![image alt >](res/Examples_Housenumbers/5.png)
![image alt >](res/Examples_Housenumbers/6.png)
![image alt >](res/Examples_Housenumbers/7.png)
![image alt >](res/Examples_Housenumbers/8.png)
![image alt >](res/Examples_Housenumbers/9.png)
<center>_Examples of the house numbers_</center>

The data-sets have a shape of: (32, 32, 3, <batchsize>). The indices are (width, height, channels, batchsize). For easier usage in Tensorflow, I changed the shape to (<batchsize>, 32, 32, 3). 

X_train.shape: 	(73257, 32, 32, 3) <br>
y_train.shape: 	(73257,) <br>
X_test.shape:  	(26032, 32, 32, 3) <br>
y_test.shape:  	(26032,)<br>

Number of training examples =	 73257 <br>
Number of testing examples =	 26032 <br>
Image data shape =		         (32, 32, 3) <br>
Number of classes =		         10 

There were no abnormalities or characteristics found, however some images are very blurry, and it is even hard for humans to identify them.
![image alt >](res/Blurry.png)
<center>_6. Example of Blurry Image_</center>


### Exploratory Visualization
In this section, you will need to provide some form of visualization that summarizes or extracts a relevant characteristic or feature about the data. The visualization should adequately support the data being used. Discuss why this visualization was chosen and how it is relevant. Questions to ask yourself when writing this section:
- _Have you visualized a relevant characteristic or feature about the dataset or input data?_
- _Is the visualization thoroughly analyzed and discussed?_
- _If a plot is provided, are the axes, title, and datum clearly defined?_



There are 10 classes with the digits from 0-9. Originally, the zeros were labeled as '10'. To remove any irritations, I changed it back to '0'.

The classes are not distributed equally. The distribution seems to follow [Benford's Law](https://www.wikiwand.com/en/Benford's_law). 


![image alt >](res/Labels_Histogram.png)
<center>_Histogram of the labels_</center>

### Algorithms and Techniques
In this section, you will need to discuss the algorithms and techniques you intend to use for solving the problem. You should justify the use of each one based on the characteristics of the problem and the problem domain. Questions to ask yourself when writing this section:
- _Are the algorithms you will use, including any default variables/parameters in the project clearly defined?_
- _Are the techniques to be used thoroughly discussed and justified?_
- _Is it made clear how the input data or datasets will be handled by the algorithms and techniques chosen?_


For solving the classification problem I will use Deep Learning techniques, to be more precise: Convolutional Neural Networks (CNN). As architecture I will use [LeNET](http://vision.stanford.edu/cs598_spring07/papers/Lecun98.pdf), which achieves good results for computer vision classification problems.

![image alt >](res/lenet.png)
<center>_LeNet Architecture_</center>


As Optimizer, I will use the AdamOptimizer. To reduce Overfitting, I will use Dropout.

**Default Parameter:**
- Normalizing = Off
- AdamOptimizer with Learning Rate = 0.001
- EPOCHS = 10
- BATCH_SIZE = 50
- Dropout Rate = 0.5

### Benchmark
In this section, you will need to provide a clearly defined benchmark result or threshold for comparing across performances obtained by your solution. The reasoning behind the benchmark (in the case where it is not an established result) should be discussed. Questions to ask yourself when writing this section:
- _Has some result or value been provided that acts as a benchmark for measuring performance?_
- _Is it clear how this result or value was obtained (whether by data or by hypothesis)?_


# <font color='red'> //TODO</font> - Erweitern?

As described, I will use **Accuracy** of the validation set as benchmark. 

## III. Methodology
_(approx. 3-5 pages)_

### Data Preprocessing
In this section, all of your preprocessing steps will need to be clearly documented, if any were necessary. From the previous section, any of the abnormalities or characteristics that you identified about the dataset will be addressed and corrected here. Questions to ask yourself when writing this section:
- _If the algorithms chosen require preprocessing steps like feature selection or feature transformations, have they been properly documented?_
- _Based on the **Data Exploration** section, if there were abnormalities or characteristics that needed to be addressed, have they been properly corrected?_
- _If no preprocessing is needed, has it been made clear why?_


For preprocessing, I used [Histogram equalization](http://docs.opencv.org/3.1.0/d5/daf/tutorial_py_histogram_equalization.html). 

_Consider an image whose pixel values are confined to some specific range of values only. For eg, brighter image will have all pixels confined to high values. But a good image will have pixels from all regions of the image. So you need to stretch this histogram to either ends (as given in below image, from wikipedia) and that is what Histogram Equalization does (in simple words). This normally improves the contrast of the image._ (from http://docs.opencv.org/3.1.0/d5/daf/tutorial_py_histogram_equalization.html)

![image alt >](res/Histogram_Equalization.png)
<center>_6. Example of Blurry Image_</center>


In addition to that, I used [One Hot Encoding](https://www.wikiwand.com/en/One-hot) for the labels. The workings is well explained in this image:

![image alt >](res/OneHot.jpg)
<center>_6. Example of Blurry Image_</center>

As another way to prevent Overfitting, I splitted the training set into 2 seperate sets: Training and Validation set.<br>
Train size: 		(49082, 32, 32, 3) <br>
Validiation size: 	(24175, 32, 32, 3) <br>
Test size: 		(26032, 32, 32, 3)

### Implementation
In this section, the process for which metrics, algorithms, and techniques that you implemented for the given data will need to be clearly documented. It should be abundantly clear how the implementation was carried out, and discussion should be made regarding any complications that occurred during this process. Questions to ask yourself when writing this section:
- _Is it made clear how the algorithms and techniques were implemented with the given datasets or input data?_
- _Were there any complications with the original metrics or techniques that required changing prior to acquiring a solution?_
- _Was there any part of the coding process (e.g., writing complicated functions) that should be documented?_


# <font color='red'> //TODO</font> - Code als Erläuterung lassen?


### Refinement
In this section, you will need to discuss the process of improvement you made upon the algorithms and techniques you used in your implementation. For example, adjusting parameters for certain models to acquire improved solutions would fall under the refinement category. Your initial and final solutions should be reported, as well as any significant intermediate results as necessary. Questions to ask yourself when writing this section:
- _Has an initial solution been found and clearly reported?_
- _Is the process of improvement clearly documented, such as what techniques were used?_
- _Are intermediate and final solutions clearly reported as the process is improved?_



## IV. Results
_(approx. 2-3 pages)_

### Model Evaluation and Validation
In this section, the final model and any supporting qualities should be evaluated in detail. It should be clear how the final model was derived and why this model was chosen. In addition, some type of analysis should be used to validate the robustness of this model and its solution, such as manipulating the input data or environment to see how the model’s solution is affected (this is called sensitivity analysis). Questions to ask yourself when writing this section:
- _Is the final model reasonable and aligning with solution expectations? Are the final parameters of the model appropriate?_
- _Has the final model been tested with various inputs to evaluate whether the model generalizes well to unseen data?_
- _Is the model robust enough for the problem? Do small perturbations (changes) in training data or the input space greatly affect the results?_
- _Can results found from the model be trusted?_


**1) EPOCHS**

| **EPOCHS** | Learning_Rate  | BATCH_SIZE | Dropout Rate | Normalisation | Training Acc | Validation Acc | Best score
|------|------|------|------|------|------|
|   **10**  | 0.001 | 50 | 0.5 | ON | 0.857 | 0.838 |
|   **25**  | 0.001 | 50 | 0.5 | ON | 0.884 | 0.856 |
|   **50**  | 0.001 | 50 | 0.5 | ON | 0.890 | 0.862 |
|   **75**  | 0.001 | 50 | 0.5 | ON | 0.908 | 0.870 | **Best** |
|   **100**  | 0.001 | 50 | 0.5 | ON | 0.904 | 0.867 |

**Result: Overfitting is visible above 75 epochs.**

**2,3) Learning Rate**

| EPOCHS | **Learning_Rate**  | BATCH_SIZE | Dropout Rate | Normalisation | Training Acc | Validation Acc | Best score
|------|------|------|------|------|------|
|   10  | **0.0005** | 50 | 0.5 | ON | 0.865 | 0.861 |
|   25  | **0.0005** | 50 | 0.5 | ON | 0.857 | 0.838 |
|   10  | **0.0002** | 50 | 0.5 | ON | 0.887 | 0.866 |
|   25  | **0.0002** | 50 | 0.5 | ON | 0.919 | 0.883 | **Best** |

**Result: Learning Rate: The lower, the better.**


**4,5) Batch_Size**

| EPOCHS | Learning_Rate  | **BATCH_SIZE** | Dropout Rate | Normalisation | Training Acc | Validation Acc | Best score
|------|------|------|------|------|------|
|   10  | 0.0002 | **25** | 0.5 | ON | 0.913 | 0.890 |
|   25  | 0.0002 | **25** | 0.5 | ON | 0.951 | 0.912 | **Best all time**
|   10  | 0.0002 | **75** | 0.5 | ON | 0.904 | 0.883 |
|   25  | 0.0002 | **75** | 0.5 | ON | 0.934 | 0.898 |

**Result: Batch Size: The lower, the better.**

**6,7) Dropout Rate**

| EPOCHS | Learning_Rate  | BATCH_SIZE | **Dropout Rate** | Normalisation | Training Acc | Validation Acc | Best score
|------|------|------|------|------|------|
|   10  | 0.0002 | 25 | **0.25** | ON | 0.850 | 0.840 |
|   25  | 0.0002 | 25 | **0.25** | ON | 0.867 | 0.853 |
|   10  | 0.0002 | 25 | **0.5** | ON | 0.913 | 0.890 |
|   25  | 0.0002 | 25 | **0.5** | ON | 0.951 | 0.912 | **Best all time**
|   10  | 0.0002 | 25 | **0.75** | ON | 0.922 | 0.888 |
|   25  | 0.0002 | 25 | **0.75** | ON | 0.960 | 0.906 |

**Result: Droprate 0.5 was best.**

**8) Normalisation**

| EPOCHS | Learning_Rate  | BATCH_SIZE | Dropout Rate | **Normalisation** | Training Acc | Validation Acc | Best score
|------|------|------|------|------|------|
|   10  | 0.0002 | 25 | 0.5 | **OFF** | 0.886 | 0.867 |
|   25  | 0.0002 | 25 | 0.5 | **OFF** | 0.924 | 0.890 |
|   10  | 0.0002 | 25 | 0.5 | **ON** | 0.913 | 0.890 |
|   25  | 0.0002 | 25 | 0.5 | **ON** | 0.951 | 0.912 | **Best all time**

**Result: Without Normalisation is worse.**

### Justification
In this section, your model’s final solution and its results should be compared to the benchmark you established earlier in the project using some type of statistical analysis. You should also justify whether these results and the solution are significant enough to have solved the problem posed in the project. Questions to ask yourself when writing this section:
- _Are the final results found stronger than the benchmark result reported earlier?_
- _Have you thoroughly analyzed and discussed the final solution?_
- _Is the final solution significant enough to have solved the problem?_


## V. Conclusion
_(approx. 1-2 pages)_

### Free-Form Visualization
In this section, you will need to provide some form of visualization that emphasizes an important quality about the project. It is much more free-form, but should reasonably support a significant result or characteristic about the problem that you want to discuss. Questions to ask yourself when writing this section:
- _Have you visualized a relevant or important quality about the problem, dataset, input data, or results?_
- _Is the visualization thoroughly analyzed and discussed?_
- _If a plot is provided, are the axes, title, and datum clearly defined?_


### Reflection
In this section, you will summarize the entire end-to-end problem solution and discuss one or two particular aspects of the project you found interesting or difficult. You are expected to reflect on the project as a whole to show that you have a firm understanding of the entire process employed in your work. Questions to ask yourself when writing this section:
- _Have you thoroughly summarized the entire process you used for this project?_
- _Were there any interesting aspects of the project?_
- _Were there any difficult aspects of the project?_
- _Does the final model and solution fit your expectations for the problem, and should it be used in a general setting to solve these types of problems?_


### Improvement
In this section, you will need to provide discussion as to how one aspect of the implementation you designed could be improved. As an example, consider ways your implementation can be made more general, and what would need to be modified. You do not need to make this improvement, but the potential solutions resulting from these changes are considered and compared/contrasted to your current solution. Questions to ask yourself when writing this section:
- _Are there further improvements that could be made on the algorithms or techniques you used in this project?_
- _Were there algorithms or techniques you researched that you did not know how to implement, but would consider using if you knew how?_
- _If you used your final solution as the new benchmark, do you think an even better solution exists?_


-----------

**Before submitting, ask yourself. . .**

- Does the project report you’ve written follow a well-organized structure similar to that of the project template?
- Is each section (particularly **Analysis** and **Methodology**) written in a clear, concise and specific fashion? Are there any ambiguous terms or phrases that need clarification?
- Would the intended audience of your project be able to understand your analysis, methods, and results?
- Have you properly proof-read your project report to assure there are minimal grammatical and spelling mistakes?
- Are all the resources used for this project correctly cited and referenced?
- Is the code that implements your solution easily readable and properly commented?
- Does the code execute without error and produce results similar to those reported?